In [4]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [5]:
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [12]:
class ConvBNRelu(tf.keras.Model):
    def __init__(self, ch, kernel_size=3, strides=1, padding='same'):
        super(ConvBNRelu, self).__init__()
        self.model = tf.keras.models.Sequential([
            keras.layers.Conv2D(ch, kernel_size=kernel_size, strides=strides, padding=padding),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu')            
        ])
        
    def call(self, x):
        x = self.model(x, training=False) # 在training=False时，BN通过整个训练集计算均值、方差去做归一化，反之，通过当前batch的方式。推理时training=False效果好。
        return x

In [13]:
class InceptionBlk(tf.keras.Model):
    def __init__(self, ch, strides=1):
        super(InceptionBlk, self).__init__()
        self.ch = ch
        self.strides = strides
        self.c1 = ConvBNRelu(ch, kernel_size=1)
        self.c2_1 = ConvBNRelu(ch, kernel_size=1)
        self.c2_2 = ConvBNRelu(ch, kernel_size=3)
        self.c3_1 = ConvBNRelu(ch, kernel_size=1)
        self.c3_2 = ConvBNRelu(ch, kernel_size=5)
        self.c4_1 = keras.layers.MaxPool2D(pool_size=(3, 3), strides=1, padding='same')
        self.c4_2 = ConvBNRelu(ch, kernel_size=1)
        
    def call(self, x):
        x1 = self.c1(x)
        x2_1 = self.c2_1(x)
        x2_2 = self.c2_2(x2_1)
        x3_1 = self.c3_1(x)
        x3_2 = self.c3_2(x3_1)
        x4_1 = self.c4_1(x)
        x4_2 = self.c4_2(x4_1)
        x = tf.concat([x1, x2_2, x3_2, x4_2], axis=-1)
        return x

In [14]:
class Inception10(tf.keras.Model):
    def __init__(self, num_blocks, num_classes, init_ch=16, **kwargs):
        super(Inception10, self).__init__(**kwargs)
#         self.in_channels = init_ch
        self.out_channels = init_ch
        self.num_blocks = num_blocks
        self.init_ch = init_ch
        self.c1 = ConvBNRelu(self.init_ch)
        self.blocks = tf.keras.models.Sequential()
        for block_id in range(self.num_blocks):
            for layer_id in range(2):
                if layer_id == 0:
                    block = InceptionBlk(self.out_channels, strides=2)
                else:
                    block = InceptionBlk(self.out_channels, strides=1)
                self.blocks.add(block)
            self.out_channels *= 2
        self.p1 = keras.layers.GlobalAveragePooling2D()
        self.fc1 = keras.layers.Dense(num_classes, activation='softmax')
        
    def call(self, x):
        x = self.c1(x)
        x = self.blocks(x)
        x = self.p1(x)
        y = self.fc1(x)
        return y

In [15]:
model = Inception10(num_blocks=2, num_classes=10)

In [16]:
model.compile(optimizer='adam',
              loss=keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

In [18]:
history = model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test), validation_freq=1)

MemoryError: Unable to allocate 1.14 GiB for an array with shape (50000, 32, 32, 3) and data type float64

In [ ]:
# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()